<a href="https://colab.research.google.com/github/masies/CRA/blob/main/Pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install tensorflow
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
tf.enable_eager_execution()

import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://code_review_automation" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 6.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-sk7hj9mi
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-sk7hj9mi
     |████████████████████████████████| 361 kB 6.3 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 10.0 MB/s 
     |████████████████████████████████| 3.4 MB 23.5 MB/s 
     |████████████████████████████████| 3.8 MB 59.3 MB/s 
     |████████████████████████████████| 2.1 MB 64.8 MB/s 
     |████████████████████████████████| 883 kB 82.9 MB/s 
     |████████████████████████████████| 3.2 MB 64.8 MB/s 
  Created wheel for t5: filename=t5-0.9.0-py3-none-any.whl size=235483 sha256=516be2d4e8b00cfe8c4b3a618d2f0d9de66f28c0e234305

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
nq_tsv_path = {
    "train":'gs://code_review_automation/raw_data/pretraining/processed/pre-training.tsv'
}

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://code_review_automation/CodeReviewModel/TestModel.model'
vocab_path = 'gs://code_review_automation/CodeReviewModel/TestModel.vocab'

TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask

def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=False, required=True),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=False)
}

In [ ]:
def nq_dataset_fn(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds


# print("A few raw train examples...")
for ex in tfds.as_numpy(nq_dataset_fn("train").take(5)):
  print(ex)

{'input': b'<extra_id_0> <technical_language> Bind indexed elements to the<extra_id_1> collection. @param name the<extra_id_2> of the<extra_id_3> to bind<extra_id_4> param target the target bindable @param elementBinder the binder<extra_id_5> use for elements @param<extra_id_6> Type the aggregate type, may be a collection<extra_id_7> an array @param elementType<extra_id_8> type @param result<extra_id_9> destination for results <extra_id_10> <code> protected final void bindIndexed(ConfigurationPropertyName name, Bindable<?> target, AggregateElementBinder<extra_id_11> Binder, ResolvableType aggregateType, ResolvableType elementType, IndexedCollectionSupplier result<extra_id_12> {<extra_id_13> (ConfigurationPropertySource<extra_id_14> : getContext().getSources())<extra_id_15> bindIndexed(source, name, target, elementBinder, result<extra_id_16> aggregate<extra_id_17> , elementType); if (result.wasSuppl<extra_id_18> ()<extra_id_19> result.<extra_id_20> ()<extra_id_21> null) { return;<extra_

In [ ]:
def preprocessing(ds):
  def to_inputs_and_targets(ex):
        inputs = tf.strings.join([ ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
#Create a new training task
t5.data.TaskRegistry.remove('pretraining')
t5.data.TaskRegistry.add(
    "pretraining",
    t5.data.Task,
    dataset_fn=nq_dataset_fn,
    splits=["train", "validation"],
    text_preprocessor=[preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("pretraining")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 512, "targets": 512})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

/usr/local/lib/python3.7/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'<technical_language> Returns every possible list<extra_id_0> can be<extra_id_1> by choosing one element from each<extra_id_2> the given sets<extra_id_3> order; the<extra_id_4> n-ary Cartesian product<extra_id_5> of the<extra_id_6> . For example: {@<extra_id_7> Sets.<extra_id_8> esianProduct(<extra_id_9> of(1, 2), ImmutableSet.of<extra_id_10> A",<extra_id_11> ", "C")) } returns a set containing six lists:<extra_id_12> code ImmutableList.of(1, "A")} {@code ImmutableList.of(1, "B")}<extra_id_13> ImmutableList.of(1, "C<extra_id_14> }<extra_id_15> .of(2, "A<extra_id_16> }<extra_id_17> code ImmutableList.of(2,<extra_id_18> B")} {@<extra_id_19> ImmutableList.of(2, "C<extra_id_20> } The result is guaranteed to be in the<extra_id_21> traditional", lexicographical order for Cartesian products<extra_id_22> you would get from nesting<extra_id_23> loops: {@code for (<extra_id_24> b0 : sets.<extra_id_25> (0)) { for (B b1<extra_id_26> 

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam

#See https://github.com/google-research/text-to-text-transfer-transformer if you want to scale up the model
MODEL_SIZE = "small"  

MODEL_DIR = 'gs://code_review_automation/model_dumps'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 512, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    learning_rate_schedule = learning_rate_schedule_noam,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None
)

In [ ]:
!gsutil cp gs://code_review_automation/CodeReviewModel/operative_config.gin ./operative_config.gin 
PATH_GIN_FILE = './operative_config.gin'
import gin
with gin.unlock_config():    
    gin.parse_config_file(PATH_GIN_FILE)
    TRAIN_STEPS = 200000
    model.train("pretraining", steps=TRAIN_STEPS)

Copying gs://code_review_automation/CodeReviewModel/operative_config.gin...
/ [1 files][ 11.5 KiB/ 11.5 KiB]                                                
Operation completed over 1 objects/11.5 KiB.                                     
INFO:tensorflow:Using config: {'_model_dir': 'gs://code_review_automation/model_dumps', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.65.126.74:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_